In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats as stats
from google.colab import files

In [2]:
premier_match = pd.read_excel("/content/PremierMatches.xlsx")
premier_ranking = pd.read_excel("/content/Premier_Teams_Ranking_Enhanced.xlsx")
premier_performance = pd.read_excel("/content/2014-2024 Premier League Teams Performance.xlsx")

In [3]:
#for all 3  dataframe, keep only the 2014-2015 to 2023-2024 season data

premier_match = premier_match[premier_match['Season'].between('2013-2014', '2023-2024')]
premier_ranking = premier_ranking[premier_ranking['Season'].between('2014-2015', '2023-2024')]
premier_performance = premier_performance[premier_performance['Season'].between('2014-2015', '2023-2024')]

In [4]:
premier_match

,Season,Date,Home Team,Score,Away,Game Outcome
0,2023-2024,2023-08-11,Burnley,0–3,Manchester City,Loss
1,2023-2024,2023-08-12,Arsenal,2–1,Nottingham Forest,Win
2,2023-2024,2023-08-12,Everton,0–1,Fulham,Loss
3,2023-2024,2023-08-12,Sheffield United,0–1,Crystal Palace,Loss
4,2023-2024,2023-08-12,Brighton,4–1,Luton Town,Win
...,...,...,...,...,...,...
4175,2013-2014,2014-05-11,Cardiff City,1–2,Chelsea,Loss
4176,2013-2014,2014-05-11,West Brom,1–2,Stoke City,Loss
4177,2013-2014,2014-05-11,Tottenham,3–0,Aston Villa,Win
4178,2013-2014,2014-05-11,Norwich City,0–2,Arsenal,Loss


In [5]:
premier_match = premier_match.rename(columns={"Home Team": "Home"})

In [6]:
# Give premier_match another column, name it as last season, and input in each cell as the previous season of indicated in Season, the example format is 2023-2024, so the last season should be 2022-2023

# Assuming 'Season' column exists and contains strings like '2023-2024'
premier_match['Last Season'] = premier_match['Season'].str.slice(0, 4).astype(int) - 1
premier_match['Last Season'] = premier_match['Last Season'].astype(str) + "-" + (premier_match['Last Season'].astype(int) + 1).astype(str)
premier_match

,Season,Date,Home,Score,Away,Game Outcome,Last Season
0,2023-2024,2023-08-11,Burnley,0–3,Manchester City,Loss,2022-2023
1,2023-2024,2023-08-12,Arsenal,2–1,Nottingham Forest,Win,2022-2023
2,2023-2024,2023-08-12,Everton,0–1,Fulham,Loss,2022-2023
3,2023-2024,2023-08-12,Sheffield United,0–1,Crystal Palace,Loss,2022-2023
4,2023-2024,2023-08-12,Brighton,4–1,Luton Town,Win,2022-2023
...,...,...,...,...,...,...,...
4175,2013-2014,2014-05-11,Cardiff City,1–2,Chelsea,Loss,2012-2013
4176,2013-2014,2014-05-11,West Brom,1–2,Stoke City,Loss,2012-2013
4177,2013-2014,2014-05-11,Tottenham,3–0,Aston Villa,Win,2012-2013
4178,2013-2014,2014-05-11,Norwich City,0–2,Arsenal,Loss,2012-2013


In [7]:
# See if home or away teams in premier_match appeared in last season, using premier_match, add  two columns, one to indicate whether home team in last season, one to indicate whether away team in last season

# Step 1: Create a mapping of seasons to the set of teams that played in each season
teams_by_season = {}

# Get unique seasons from 'Season' and 'Last Season' columns
all_seasons = pd.unique(premier_match[['Season', 'Last Season']].values.ravel('K'))

for season in all_seasons:
    # Get matches for the current season
    season_matches = premier_match[premier_match['Season'] == season]

    # If there are no matches for the season (possible for 'Last Season' that isn't in 'Season'), skip
    if season_matches.empty:
        teams_by_season[season] = set()
        continue

    # Get unique home and away teams
    home_teams = season_matches['Home'].unique()
    away_teams = season_matches['Away'].unique()

    # Combine home and away teams into a set
    teams = set(home_teams).union(set(away_teams))

    # Store the set of teams for the current season
    teams_by_season[season] = teams

# Step 2: Define a function to check if teams were in the last season
def check_team_in_last_season(row):
    home_team = row['Home']
    away_team = row['Away']
    last_season = row['Last Season']

    # Get the set of teams from the last season
    teams_last_season = teams_by_season.get(last_season, set())

    # Check if the home and away teams were in the last season
    home_in_last_season = home_team in teams_last_season
    away_in_last_season = away_team in teams_last_season

    return pd.Series({
        'HomeTeamInLastSeason': home_in_last_season,
        'AwayTeamInLastSeason': away_in_last_season
    })

# Step 3: Apply the function to the DataFrame
premier_match[['HomeTeamInLastSeason', 'AwayTeamInLastSeason']] = premier_match.apply(
    check_team_in_last_season, axis=1
)

# Now 'premier_match' has two new columns indicating whether home and away teams were in the last season
premier_match

,Season,Date,Home,Score,Away,Game Outcome,Last Season,HomeTeamInLastSeason,AwayTeamInLastSeason
0,2023-2024,2023-08-11,Burnley,0–3,Manchester City,Loss,2022-2023,False,True
1,2023-2024,2023-08-12,Arsenal,2–1,Nottingham Forest,Win,2022-2023,True,True
2,2023-2024,2023-08-12,Everton,0–1,Fulham,Loss,2022-2023,True,True
3,2023-2024,2023-08-12,Sheffield United,0–1,Crystal Palace,Loss,2022-2023,False,True
4,2023-2024,2023-08-12,Brighton,4–1,Luton Town,Win,2022-2023,True,False
...,...,...,...,...,...,...,...,...,...
4175,2013-2014,2014-05-11,Cardiff City,1–2,Chelsea,Loss,2012-2013,False,False
4176,2013-2014,2014-05-11,West Brom,1–2,Stoke City,Loss,2012-2013,False,False
4177,2013-2014,2014-05-11,Tottenham,3–0,Aston Villa,Win,2012-2013,False,False
4178,2013-2014,2014-05-11,Norwich City,0–2,Arsenal,Loss,2012-2013,False,False


In [8]:
 promoted_teams = premier_match[premier_match['HomeTeamInLastSeason'] == False].groupby('Season')['Home'].unique()
 promoted_teams

,Home
Season,
2013-2014,"[Liverpool, Norwich City, Swansea City, West H..."
2014-2015,"[Leicester City, QPR, Burnley]"
2015-2016,"[Norwich City, Bournemouth, Watford]"
2016-2017,"[Hull City, Burnley, Middlesbrough]"
2017-2018,"[Brighton, Newcastle United, Huddersfield]"
2018-2019,"[Fulham, Wolves, Cardiff City]"
2019-2020,"[Norwich City, Aston Villa, Sheffield United]"
2020-2021,"[Fulham, West Brom, Leeds United]"
2021-2022,"[Brentford, Watford, Norwich City]"


In [9]:
# Dictionary of promoted teams per season
promoted_teams = {
    '2014-2015': ['Leicester City', 'QPR', 'Burnley'],
    '2015-2016': ['Norwich City', 'Bournemouth', 'Watford'],
    '2016-2017': ['Hull City', 'Burnley', 'Middlesbrough'],
    '2017-2018': ['Brighton', 'Newcastle United', 'Huddersfield'],
    '2018-2019': ['Fulham', 'Wolves', 'Cardiff City'],
    '2019-2020': ['Norwich City', 'Aston Villa', 'Sheffield United'],
    '2020-2021': ['Fulham', 'West Brom', 'Leeds United'],
    '2021-2022': ['Brentford', 'Watford', 'Norwich City'],
    '2022-2023': ['Fulham', 'Bournemouth', 'Nottingham Forest'],
    '2023-2024': ['Burnley', 'Sheffield United', 'Luton Town']
}

In [10]:
# Initialize a dictionary to store the average possession of promoted teams
avg_poss_promoted_by_season = {}

# Get a sorted list of seasons to maintain chronological order
seasons = sorted(promoted_teams.keys())

# Loop through each season to calculate the average possession of promoted teams
for idx, season in enumerate(seasons):
    if idx == 0:
        # For the first season, there is no previous season to reference
        avg_poss_promoted_by_season[season] = np.nan
    else:
        last_season = seasons[idx - 1]
        # Get the promoted teams from the last season
        teams = promoted_teams[last_season]

        # Get possession data for these teams in the last season (their first season in the Premier League)
        team_stats = premier_performance[
            (premier_performance['Season'] == last_season) &
            (premier_performance['Squad'].isin(teams))
        ]

        # Calculate the average possession
        avg_poss = team_stats['Poss'].mean()
        avg_poss_promoted_by_season[season] = avg_poss

# Create a dictionary to map (team, season) to possession percentage
team_poss_dict = {}

for index, row in premier_performance.iterrows():
    team_name = row['Squad']
    season = row['Season']
    poss = row['Poss']
    team_poss_dict[(team_name, season)] = poss

def get_last_season_poss(team, last_season, in_last_season, current_season):
    if in_last_season:
        # Team was in the last season, retrieve actual possession
        if (team, last_season) in team_poss_dict:
            return team_poss_dict[(team, last_season)]
        else:
            return np.nan  # No data found for the team and last season
    else:
        # Team was promoted this season, assign average possession of promoted teams from previous season
        avg_poss = avg_poss_promoted_by_season.get(current_season)
        if pd.notna(avg_poss):
            return avg_poss
        else:
            return 0  # Default value if no average is available

# Apply the function to create the new columns
premier_match['Last Season Poss Home'] = premier_match.apply(
    lambda row: get_last_season_poss(
        row['Home'],             # Home team
        row['Last Season'],      # Last season
        row['HomeTeamInLastSeason'],  # Boolean if home team was in last season
        row['Season']            # Current season
    ), axis=1)

premier_match['Last Season Poss Away'] = premier_match.apply(
    lambda row: get_last_season_poss(
        row['Away'],             # Away team
        row['Last Season'],      # Last season
        row['AwayTeamInLastSeason'],  # Boolean if away team was in last season
        row['Season']            # Current season
    ), axis=1)


In [11]:
# Initialize a dictionary to store the average possession of promoted teams
avg_GF_promoted_by_season = {}

# Get a sorted list of seasons to maintain chronological order
seasons = sorted(promoted_teams.keys())

# Loop through each season to calculate the average possession of promoted teams
for idx, season in enumerate(seasons):
    if idx == 0:
        # For the first season, there is no previous season to reference
        avg_GF_promoted_by_season[season] = np.nan
    else:
        last_season = seasons[idx - 1]
        # Get the promoted teams from the last season
        teams = promoted_teams[last_season]

        # Get possession data for these teams in the last season (their first season in the Premier League)
        team_stats = premier_ranking[
            (premier_ranking['Season'] == last_season) &
            (premier_ranking['Team Name'].isin(teams))
        ]

        # Calculate the average possession
        avg_GF = team_stats['Goals For'].mean()
        avg_GF_promoted_by_season[season] = avg_GF

# Create a dictionary to map (team, season) to possession percentage
team_GF_dict = {}

for index, row in premier_ranking.iterrows():
    team_name = row['Team Name']
    season = row['Season']
    GF = row['Goals For']
    team_GF_dict[(team_name, season)] = GF

def get_last_season_GF(team, last_season, in_last_season, current_season):
    if in_last_season:
        # Team was in the last season, retrieve actual possession
        if (team, last_season) in team_GF_dict:
            return team_GF_dict[(team, last_season)]
        else:
            return np.nan  # No data found for the team and last season
    else:
        # Team was promoted this season, assign average possession of promoted teams from previous season
        avg_GF = avg_GF_promoted_by_season.get(current_season)
        if pd.notna(avg_GF):
            return avg_GF
        else:
            return 0  # Default value if no average is available

# Apply the function to create the new columns
premier_match['Last Season GF Home'] = premier_match.apply(
    lambda row: get_last_season_GF(
        row['Home'],             # Home team
        row['Last Season'],      # Last season
        row['HomeTeamInLastSeason'],  # Boolean if home team was in last season
        row['Season']            # Current season
    ), axis=1)

premier_match['Last Season GF Away'] = premier_match.apply(
    lambda row: get_last_season_GF(
        row['Away'],             # Away team
        row['Last Season'],      # Last season
        row['AwayTeamInLastSeason'],  # Boolean if away team was in last season
        row['Season']            # Current season
    ), axis=1)


In [12]:
# Initialize a dictionary to store the average possession of promoted teams
avg_GA_promoted_by_season = {}

# Get a sorted list of seasons to maintain chronological order
seasons = sorted(promoted_teams.keys())

# Loop through each season to calculate the average possession of promoted teams
for idx, season in enumerate(seasons):
    if idx == 0:
        # For the first season, there is no previous season to reference
        avg_GA_promoted_by_season[season] = np.nan
    else:
        last_season = seasons[idx - 1]
        # Get the promoted teams from the last season
        teams = promoted_teams[last_season]

        # Get possession data for these teams in the last season (their first season in the Premier League)
        team_stats = premier_ranking[
            (premier_ranking['Season'] == last_season) &
            (premier_ranking['Team Name'].isin(teams))
        ]

        # Calculate the average possession
        avg_GA = team_stats['Goals Against'].mean()
        avg_GA_promoted_by_season[season] = avg_GA

# Create a dictionary to map (team, season) to possession percentage
team_GA_dict = {}

for index, row in premier_ranking.iterrows():
    team_name = row['Team Name']
    season = row['Season']
    GA = row['Goals Against']
    team_GA_dict[(team_name, season)] = GA

def get_last_season_GA(team, last_season, in_last_season, current_season):
    if in_last_season:
        # Team was in the last season, retrieve actual possession
        if (team, last_season) in team_GA_dict:
            return team_GA_dict[(team, last_season)]
        else:
            return np.nan  # No data found for the team and last season
    else:
        # Team was promoted this season, assign average possession of promoted teams from previous season
        avg_GA = avg_GA_promoted_by_season.get(current_season)
        if pd.notna(avg_GA):
            return avg_GA
        else:
            return 0  # Default value if no average is available

# Apply the function to create the new columns
premier_match['Last Season GA Home'] = premier_match.apply(
    lambda row: get_last_season_GA(
        row['Home'],             # Home team
        row['Last Season'],      # Last season
        row['HomeTeamInLastSeason'],  # Boolean if home team was in last season
        row['Season']            # Current season
    ), axis=1)

premier_match['Last Season GA Away'] = premier_match.apply(
    lambda row: get_last_season_GA(
        row['Away'],             # Away team
        row['Last Season'],      # Last season
        row['AwayTeamInLastSeason'],  # Boolean if away team was in last season
        row['Season']            # Current season
    ), axis=1)


In [13]:
# Initialize a dictionary to store the average possession of promoted teams
avg_yellowcard_promoted_by_season = {}

# Get a sorted list of seasons to maintain chronological order
seasons = sorted(promoted_teams.keys())

# Loop through each season to calculate the average possession of promoted teams
for idx, season in enumerate(seasons):
    if idx == 0:
        # For the first season, there is no previous season to reference
        avg_yellowcard_promoted_by_season[season] = np.nan
    else:
        last_season = seasons[idx - 1]
        # Get the promoted teams from the last season
        teams = promoted_teams[last_season]

        # Get possession data for these teams in the last season (their first season in the Premier League)
        team_stats = premier_performance[
            (premier_performance['Season'] == last_season) &
            (premier_performance['Squad'].isin(teams))
        ]

        # Calculate the average possession
        avg_yellowcard = team_stats['CrdY'].mean()
        avg_yellowcard_promoted_by_season[season] = avg_yellowcard

# Create a dictionary to map (team, season) to possession percentage
team_yellowcard_dict = {}

for index, row in premier_performance.iterrows():
    team_name = row['Squad']
    season = row['Season']
    CrdY = row['CrdY']
    team_yellowcard_dict[(team_name, season)] = CrdY

def get_last_season_yellowcard(team, last_season, in_last_season, current_season):
    if in_last_season:
        # Team was in the last season, retrieve actual possession
        if (team, last_season) in team_yellowcard_dict:
            return team_yellowcard_dict[(team, last_season)]
        else:
            return np.nan  # No data found for the team and last season
    else:
        # Team was promoted this season, assign average possession of promoted teams from previous season
        avg_yellowcard = avg_yellowcard_promoted_by_season.get(current_season)
        if pd.notna(avg_yellowcard):
            return avg_yellowcard
        else:
            return 0  # Default value if no average is available

# Apply the function to create the new columns
premier_match['Last Season Yellow Card Home'] = premier_match.apply(
    lambda row: get_last_season_yellowcard(
        row['Home'],             # Home team
        row['Last Season'],      # Last season
        row['HomeTeamInLastSeason'],  # Boolean if home team was in last season
        row['Season']            # Current season
    ), axis=1)

premier_match['Last Season Yellow Cards Away'] = premier_match.apply(
    lambda row: get_last_season_yellowcard(
        row['Away'],             # Away team
        row['Last Season'],      # Last season
        row['AwayTeamInLastSeason'],  # Boolean if away team was in last season
        row['Season']            # Current season
    ), axis=1)


In [14]:
# Initialize a dictionary to store the average possession of promoted teams
avg_penalty_promoted_by_season = {}

# Get a sorted list of seasons to maintain chronological order
seasons = sorted(promoted_teams.keys())

# Loop through each season to calculate the average possession of promoted teams
for idx, season in enumerate(seasons):
    if idx == 0:
        # For the first season, there is no previous season to reference
        avg_penalty_promoted_by_season[season] = np.nan
    else:
        last_season = seasons[idx - 1]
        # Get the promoted teams from the last season
        teams = promoted_teams[last_season]

        # Get possession data for these teams in the last season (their first season in the Premier League)
        team_stats = premier_performance[
            (premier_performance['Season'] == last_season) &
            (premier_performance['Squad'].isin(teams))
        ]

        # Calculate the average possession
        avg_penalty = team_stats['Penalty Kicks Made'].mean()
        avg_penalty_promoted_by_season[season] = avg_penalty

# Create a dictionary to map (team, season) to possession percentage
team_penalty_dict = {}

for index, row in premier_performance.iterrows():
    team_name = row['Squad']
    season = row['Season']
    Penalty = row['Penalty Kicks Made']
    team_penalty_dict[(team_name, season)] = Penalty

def get_last_season_penalty(team, last_season, in_last_season, current_season):
    if in_last_season:
        # Team was in the last season, retrieve actual possession
        if (team, last_season) in team_penalty_dict:
            return team_penalty_dict[(team, last_season)]
        else:
            return np.nan  # No data found for the team and last season
    else:
        # Team was promoted this season, assign average possession of promoted teams from previous season
        avg_penalty = avg_penalty_promoted_by_season.get(current_season)
        if pd.notna(avg_penalty):
            return avg_penalty
        else:
            return 0  # Default value if no average is available

# Apply the function to create the new columns
premier_match['Last Season Penalty Home'] = premier_match.apply(
    lambda row: get_last_season_penalty(
        row['Home'],             # Home team
        row['Last Season'],      # Last season
        row['HomeTeamInLastSeason'],  # Boolean if home team was in last season
        row['Season']            # Current season
    ), axis=1)

premier_match['Last Season Penalty Away'] = premier_match.apply(
    lambda row: get_last_season_penalty(
        row['Away'],             # Away team
        row['Last Season'],      # Last season
        row['AwayTeamInLastSeason'],  # Boolean if away team was in last season
        row['Season']            # Current season
    ), axis=1)


In [15]:
premier_match

,Season,Date,Home,Score,Away,Game Outcome,Last Season,HomeTeamInLastSeason,AwayTeamInLastSeason,Last Season Poss Home,Last Season Poss Away,Last Season GF Home,Last Season GF Away,Last Season GA Home,Last Season GA Away,Last Season Yellow Card Home,Last Season Yellow Cards Away,Last Season Penalty Home,Last Season Penalty Away
0,2023-2024,2023-08-11,Burnley,0–3,Manchester City,Loss,2022-2023,False,True,42.266667,64.700000,43.333333,94.000000,64.0,33.0,77.333333,44.000000,2.666667,9.000000
1,2023-2024,2023-08-12,Arsenal,2–1,Nottingham Forest,Win,2022-2023,True,True,59.300000,37.600000,88.000000,38.000000,43.0,68.0,51.000000,84.000000,3.000000,3.000000
2,2023-2024,2023-08-12,Everton,0–1,Fulham,Loss,2022-2023,True,True,42.800000,48.800000,34.000000,55.000000,57.0,53.0,81.000000,80.000000,3.000000,5.000000
3,2023-2024,2023-08-12,Sheffield United,0–1,Crystal Palace,Loss,2022-2023,False,True,42.266667,46.300000,43.333333,40.000000,64.0,49.0,77.333333,82.000000,2.666667,1.000000
4,2023-2024,2023-08-12,Brighton,4–1,Luton Town,Win,2022-2023,True,False,60.200000,42.266667,72.000000,43.333333,53.0,64.0,58.000000,77.333333,6.000000,2.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,2013-2014,2014-05-11,Cardiff City,1–2,Chelsea,Loss,2012-2013,False,False,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
4176,2013-2014,2014-05-11,West Brom,1–2,Stoke City,Loss,2012-2013,False,False,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
4177,2013-2014,2014-05-11,Tottenham,3–0,Aston Villa,Win,2012-2013,False,False,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
4178,2013-2014,2014-05-11,Norwich City,0–2,Arsenal,Loss,2012-2013,False,False,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [16]:
# Step 1: Convert 'Date' to datetime
premier_match['Date'] = pd.to_datetime(premier_match['Date'])

# Step 2: Create a unified DataFrame for all matches from each team's perspective
# a. Home team perspective
home_df = premier_match[['Date', 'Home', 'Away', 'Game Outcome']].copy()
home_df.rename(columns={'Home': 'Team', 'Away': 'Opponent'}, inplace=True)
home_df['Points'] = home_df['Game Outcome'].map({'Win': 3, 'Draw': 1, 'Loss': 0})

# b. Away team perspective
away_df = premier_match[['Date', 'Away', 'Home', 'Game Outcome']].copy()

# Invert the 'Game Outcome' for the away team
def invert_outcome(outcome):
    if outcome == 'Win':
        return 'Loss'
    elif outcome == 'Loss':
        return 'Win'
    else:
        return 'Draw'

away_df['Game Outcome'] = away_df['Game Outcome'].apply(invert_outcome)
away_df.rename(columns={'Away': 'Team', 'Home': 'Opponent'}, inplace=True)
away_df['Points'] = away_df['Game Outcome'].map({'Win': 3, 'Draw': 1, 'Loss': 0})

# Combine home and away data
team_matches = pd.concat([home_df, away_df], ignore_index=True)

# Step 3: Sort by Team and Date
team_matches.sort_values(['Team', 'Date'], inplace=True)

# Step 4: Shift 'Points' column to exclude current match
team_matches['Points_Shifted'] = team_matches.groupby('Team')['Points'].shift(1)
team_matches['Points_Shifted'] = team_matches['Points_Shifted'].fillna(0)

# Step 5: Calculate rolling sum over the last 5 games for each team
team_matches['Recent5Points'] = team_matches.groupby('Team')['Points_Shifted'].apply(
    lambda x: x.rolling(window=5, min_periods=1).sum()
).reset_index(level=0, drop=True)

# Step 6: Merge Recent5Points back into the original DataFrame for both Home and Away teams

# a. Recent5Points for Home teams
home_recent_points = team_matches[['Date', 'Team', 'Recent5Points']].copy()
home_recent_points.rename(columns={'Team': 'Home', 'Recent5Points': 'HomeRecent5Points'}, inplace=True)

# b. Recent5Points for Away teams
away_recent_points = team_matches[['Date', 'Team', 'Recent5Points']].copy()
away_recent_points.rename(columns={'Team': 'Away', 'Recent5Points': 'AwayRecent5Points'}, inplace=True)

# Step 7: Merge the recent points into the original DataFrame
premier_match = premier_match.merge(home_recent_points, on=['Date', 'Home'], how='left')
premier_match = premier_match.merge(away_recent_points, on=['Date', 'Away'], how='left')

# Step 8: Display the updated DataFrame
premier_match

,Season,Date,Home,Score,Away,Game Outcome,Last Season,HomeTeamInLastSeason,AwayTeamInLastSeason,Last Season Poss Home,...,Last Season GF Home,Last Season GF Away,Last Season GA Home,Last Season GA Away,Last Season Yellow Card Home,Last Season Yellow Cards Away,Last Season Penalty Home,Last Season Penalty Away,HomeRecent5Points,AwayRecent5Points
0,2023-2024,2023-08-11,Burnley,0–3,Manchester City,Loss,2022-2023,False,True,42.266667,...,43.333333,94.000000,64.0,33.0,77.333333,44.000000,2.666667,9.000000,4.0,10.0
1,2023-2024,2023-08-12,Arsenal,2–1,Nottingham Forest,Win,2022-2023,True,True,59.300000,...,88.000000,38.000000,43.0,68.0,51.000000,84.000000,3.000000,3.000000,9.0,8.0
2,2023-2024,2023-08-12,Everton,0–1,Fulham,Loss,2022-2023,True,True,42.800000,...,34.000000,55.000000,57.0,53.0,81.000000,80.000000,3.000000,5.000000,8.0,7.0
3,2023-2024,2023-08-12,Sheffield United,0–1,Crystal Palace,Loss,2022-2023,False,True,42.266667,...,43.333333,40.000000,64.0,49.0,77.333333,82.000000,2.666667,1.000000,6.0,8.0
4,2023-2024,2023-08-12,Brighton,4–1,Luton Town,Win,2022-2023,True,False,60.200000,...,72.000000,43.333333,53.0,64.0,58.000000,77.333333,6.000000,2.666667,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,2013-2014,2014-05-11,Cardiff City,1–2,Chelsea,Loss,2012-2013,False,False,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,4.0,10.0
4176,2013-2014,2014-05-11,West Brom,1–2,Stoke City,Loss,2012-2013,False,False,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,4.0,7.0
4177,2013-2014,2014-05-11,Tottenham,3–0,Aston Villa,Win,2012-2013,False,False,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,10.0,4.0
4178,2013-2014,2014-05-11,Norwich City,0–2,Arsenal,Loss,2012-2013,False,False,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.0,12.0


In [17]:
# Parse 'Score' to extract goals
premier_match[['HomeGoals', 'AwayGoals']] = premier_match['Score'].str.split('–', expand=True).astype(int)

# Create DataFrames for Home and Away Teams

# a. Home Team Perspective
home_df = premier_match[['Date', 'Home', 'Away', 'HomeGoals', 'AwayGoals']].copy()
home_df.rename(columns={
    'Home': 'Team',
    'Away': 'Opponent',
    'HomeGoals': 'Goals_For',
    'AwayGoals': 'Goals_Against'
}, inplace=True)

# b. Away Team Perspective
away_df = premier_match[['Date', 'Away', 'Home', 'AwayGoals', 'HomeGoals']].copy()
away_df.rename(columns={
    'Away': 'Team',
    'Home': 'Opponent',
    'AwayGoals': 'Goals_For',
    'HomeGoals': 'Goals_Against'
}, inplace=True)

# Combine Home and Away DataFrames
combined_df = pd.concat([home_df, away_df], ignore_index=True)

# Sort by Team and Date
combined_df.sort_values(['Team', 'Date'], inplace=True)

# Shift 'Goals_For' and 'Goals_Against' columns to exclude current match
combined_df['Goals_For_Shifted'] = combined_df.groupby('Team')['Goals_For'].shift(1)
combined_df['Goals_Against_Shifted'] = combined_df.groupby('Team')['Goals_Against'].shift(1)

# Do not fill NaN values with zeros
# combined_df['Goals_For_Shifted'] = combined_df['Goals_For_Shifted'].fillna(0)
# combined_df['Goals_Against_Shifted'] = combined_df['Goals_Against_Shifted'].fillna(0)

# Calculate rolling average over the last 5 games
combined_df['AvgGF_Last5'] = combined_df.groupby('Team')['Goals_For_Shifted'].apply(
    lambda x: x.rolling(window=5, min_periods=1).mean()
).reset_index(level=0, drop=True)
combined_df['AvgGA_Last5'] = combined_df.groupby('Team')['Goals_Against_Shifted'].apply(
    lambda x: x.rolling(window=5, min_periods=1).mean()
).reset_index(level=0, drop=True)

# Merge the calculated averages back into the original DataFrame for both Home and Away teams

# a. Averages for Home Teams
home_averages = combined_df[['Date', 'Team', 'AvgGF_Last5', 'AvgGA_Last5']].copy()
home_averages.rename(columns={
    'Team': 'Home',
    'AvgGF_Last5': 'HomeAvgGF_Last5',
    'AvgGA_Last5': 'HomeAvgGA_Last5'
}, inplace=True)

# b. Averages for Away Teams
away_averages = combined_df[['Date', 'Team', 'AvgGF_Last5', 'AvgGA_Last5']].copy()
away_averages.rename(columns={
    'Team': 'Away',
    'AvgGF_Last5': 'AwayAvgGF_Last5',
    'AvgGA_Last5': 'AwayAvgGA_Last5'
}, inplace=True)

# Merge back into the original DataFrame
premier_match = premier_match.merge(home_averages, on=['Date', 'Home'], how='left')
premier_match = premier_match.merge(away_averages, on=['Date', 'Away'], how='left')

# Review the updated DataFrame
premier_match


,Season,Date,Home,Score,Away,Game Outcome,Last Season,HomeTeamInLastSeason,AwayTeamInLastSeason,Last Season Poss Home,...,Last Season Penalty Home,Last Season Penalty Away,HomeRecent5Points,AwayRecent5Points,HomeGoals,AwayGoals,HomeAvgGF_Last5,HomeAvgGA_Last5,AwayAvgGF_Last5,AwayAvgGA_Last5
0,2023-2024,2023-08-11,Burnley,0–3,Manchester City,Loss,2022-2023,False,True,42.266667,...,2.666667,9.000000,4.0,10.0,0,3,1.0,1.6,1.4,0.6
1,2023-2024,2023-08-12,Arsenal,2–1,Nottingham Forest,Win,2022-2023,True,True,59.300000,...,3.000000,3.000000,9.0,8.0,2,1,2.0,1.0,1.8,1.6
2,2023-2024,2023-08-12,Everton,0–1,Fulham,Loss,2022-2023,True,True,42.800000,...,3.000000,5.000000,8.0,7.0,0,1,1.8,1.4,2.0,1.6
3,2023-2024,2023-08-12,Sheffield United,0–1,Crystal Palace,Loss,2022-2023,False,True,42.266667,...,2.666667,1.000000,6.0,8.0,0,1,0.4,1.4,1.8,1.4
4,2023-2024,2023-08-12,Brighton,4–1,Luton Town,Win,2022-2023,True,False,60.200000,...,6.000000,2.666667,7.0,0.0,4,1,1.8,1.6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,2013-2014,2014-05-11,Cardiff City,1–2,Chelsea,Loss,2012-2013,False,False,0.000000,...,0.000000,0.000000,4.0,10.0,1,2,0.4,2.2,1.4,0.4
4176,2013-2014,2014-05-11,West Brom,1–2,Stoke City,Loss,2012-2013,False,False,0.000000,...,0.000000,0.000000,4.0,7.0,1,2,1.0,1.8,1.2,1.2
4177,2013-2014,2014-05-11,Tottenham,3–0,Aston Villa,Win,2012-2013,False,False,0.000000,...,0.000000,0.000000,10.0,4.0,3,0,2.4,1.4,0.8,2.0
4178,2013-2014,2014-05-11,Norwich City,0–2,Arsenal,Loss,2012-2013,False,False,0.000000,...,0.000000,0.000000,1.0,12.0,0,2,0.4,1.8,2.0,0.8


In [18]:
# from google.colab import files

# # Save the DataFrame to an Excel file
# premier_match.to_excel('premier_match_raw.xlsx', index=False)

# # Download the file
# files.download('premier_match_raw.xlsx')